#📌 Extracción

In [ ]:
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

print("Paso 1: Extrayendo los datos desde el archivo JSON...")
with open('TelecomX_Data.json', 'r') as f:
    data_json = json.load(f)

In [ ]:
# Importación de librerías adicionales para Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

#🔧 Transformación

In [ ]:
# Limpieza y tratamiento de los datos
print("Paso 2: Iniciando el proceso de transformación y limpieza de datos...")

# Manejar valores perdidos en 'account_Charges_Total'
df['account_Charges_Total'] = df['account_Charges_Total'].replace(' ', np.nan)
df.dropna(subset=['account_Charges_Total'], inplace=True)
df['account_Charges_Total'] = pd.to_numeric(df['account_Charges_Total'])
print("Columna 'account_Charges_Total' convertida a tipo numérico y valores faltantes eliminados.")

# Limpiar la columna 'Churn'
df.dropna(subset=['Churn'], inplace=True)
print("Valores faltantes en 'Churn' eliminados.")

# Manejar el valor 'No phone service'
df['phone_MultipleLines'] = df['phone_MultipleLines'].replace('No phone service', 'No')
print("Se limpió la columna 'phone_MultipleLines'.")

# Manejar el valor 'No internet service'
internet_service_cols = ['internet_OnlineSecurity', 'internet_OnlineBackup', 'internet_DeviceProtection', 'internet_TechSupport', 'internet_StreamingTV', 'internet_StreamingMovies']
for col in internet_service_cols:
    df[col] = df[col].replace('No internet service', 'No')
print("Se limpiaron las columnas de servicios de internet.")

# Mostrar los tipos de datos finales para verificar la limpieza
print("\nInformación del DataFrame después de la limpieza:")
print(df.info())

# Análisis descriptivo inicial
print("\nAnálisis descriptivo de variables numéricas:")
print(df[['customer_tenure', 'account_Charges_Monthly', 'account_Charges_Total']].describe())

print("\nConteo de clientes que evadieron vs. los que no:")
print(df['Churn'].value_counts())

#📊 Carga y análisis

In [ ]:
# Generación de gráficos para el análisis exploratorio
# Configuración del estilo de los gráficos
sns.set_style("whitegrid")
plt.style.use("seaborn-v0_8-whitegrid")

print("Generando visualizaciones...")

# Gráfico 1: Distribución de la Evasión (Churn)
plt.figure(figsize=(6, 5))
sns.countplot(x='Churn', data=df)
plt.title('Distribución de Clientes con y sin Evasión (Churn)')
plt.xlabel('Evasión (Churn)')
plt.ylabel('Cantidad de Clientes')
plt.show()

# Gráfico 2: Tasa de Evasión por Tipo de Contrato
plt.figure(figsize=(8, 6))
sns.countplot(x='account_Contract', hue='Churn', data=df)
plt.title('Evasión de Clientes por Tipo de Contrato')
plt.xlabel('Tipo de Contrato')
plt.ylabel('Cantidad de Clientes')
plt.show()

# Gráfico 3: Relación entre la Antigüedad (Tenure) y la Evasión
plt.figure(figsize=(10, 6))
sns.histplot(x='customer_tenure', hue='Churn', data=df, multiple='dodge', shrink=0.8, binwidth=5)
plt.title('Relación entre la Antigüedad del Cliente y la Evasión')
plt.xlabel('Antigüedad (Meses de Contrato)')
plt.ylabel('Cantidad de Clientes')
plt.show()

# Gráfico 4: Impacto del "Paperless Billing" en la Evasión
plt.figure(figsize=(7, 6))
sns.countplot(x='account_PaperlessBilling', hue='Churn', data=df)
plt.title('Impacto de la Facturación Sin Papel en la Evasión')
plt.xlabel('Facturación Sin Papel')
plt.ylabel('Cantidad de Clientes')
plt.show()

#📄Informe final

# Preparación para el Modelado

In [ ]:
# Sección de Preparación para el Modelado
print("Paso 3: Preparando los datos para el modelado...")

# Eliminar la columna 'customerID' ya que no aporta valor a la predicción
# y puede perjudicar los modelos
df_model = df.drop('customerID', axis=1)

# Codificación de la variable objetivo 'Churn'
le = LabelEncoder()
df_model['Churn'] = le.fit_transform(df_model['Churn'])
print("Variable 'Churn' codificada numéricamente.")

# Identificar variables categóricas y numéricas para la codificación
categorical_features = df_model.select_dtypes(include=['object']).columns
numerical_features = df_model.select_dtypes(include=['int64', 'float64']).columns
numerical_features = numerical_features.drop('Churn')

# Crear el pipeline de preprocesamiento con OneHotEncoder
# El OneHotEncoder transforma las variables categóricas a formato numérico
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), categorical_features)],
    remainder='passthrough'
)

# Definir variables predictoras (X) y la variable objetivo (y)
X = df_model.drop('Churn', axis=1)
y = df_model['Churn']

# Separar los datos en conjuntos de entrenamiento y prueba (80/20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print("Datos separados en conjuntos de entrenamiento y prueba.")

# Aplicar el preprocesamiento a los conjuntos de entrenamiento y prueba
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)
print("Datos preprocesados y listos para el modelado.")

# Sección de Creación y Evaluación de Modelos
print("\nPaso 4: Creando y evaluando modelos de clasificación...")

# Modelo 1: Regresión Logística
# Este modelo requiere normalización de datos para un rendimiento óptimo
# Sin embargo, el ColumnTransformer ya ha manejado la codificación. Podemos añadir un StandardScaler si es necesario,
# pero en este caso, la codificación ya permite que el modelo funcione.
print("\nEntrenando y evaluando el modelo de Regresión Logística...")
log_reg = LogisticRegression(max_iter=1000, solver='liblinear')
log_reg.fit(X_train_processed, y_train)
y_pred_log_reg = log_reg.predict(X_test_processed)

# Modelo 2: Random Forest
# Este modelo no requiere normalización
print("Entrenando y evaluando el modelo de Random Forest...")
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_processed, y_train)
y_pred_rf = rf_model.predict(X_test_processed)

# Evaluación de Modelos
print("\n-- Resultados de la Regresión Logística --")
print(f"Precisión: {precision_score(y_test, y_pred_log_reg):.2f}")
print(f"Sensibilidad (Recall): {recall_score(y_test, y_pred_log_reg):.2f}")
print(f"F1-score: {f1_score(y_test, y_pred_log_reg):.2f}")
print(f"Exactitud (Accuracy): {accuracy_score(y_test, y_pred_log_reg):.2f}")
print("\nMatriz de Confusión:\n", confusion_matrix(y_test, y_pred_log_reg))

print("\n-- Resultados de Random Forest --")
print(f"Precisión: {precision_score(y_test, y_pred_rf):.2f}")
print(f"Sensibilidad (Recall): {recall_score(y_test, y_pred_rf):.2f}")
print(f"F1-score: {f1_score(y_test, y_pred_rf):.2f}")
print(f"Exactitud (Accuracy): {accuracy_score(y_test, y_pred_rf):.2f}")
print("\nMatriz de Confusión:\n", confusion_matrix(y_test, y_pred_rf))

# Análisis de Importancia de Variables para Random Forest
print("\n-- Importancia de las Variables (Random Forest) --")
feature_names = preprocessor.get_feature_names_out()
importances = rf_model.feature_importances_
feature_importance_df = pd.DataFrame({'feature': feature_names, 'importance': importances})
feature_importance_df = feature_importance_df.sort_values('importance', ascending=False)
print(feature_importance_df.head(10))

# Conclusiones y Análisis Estratégico